In [1]:
!pip install -q langchain langchain-groq langchain-core pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 4.1 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
api_key = userdata.get('GROQ_API_KEY')

In [3]:
from langchain_groq import ChatGroq
chat = ChatGroq(api_key = api_key, model = "llama-3.3-70b-versatile")

In [4]:
chat.invoke("hi") #testing the model instantiation

AIMessage(content="It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 36, 'total_tokens': 59, 'completion_time': 0.084145689, 'prompt_time': 0.004437277, 'queue_time': 0.23458809700000002, 'total_time': 0.088582966}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_7b42aeb9fa', 'finish_reason': 'stop', 'logprobs': None}, id='run-ce7ed68d-842a-49fc-b80f-7898b8d01f8a-0', usage_metadata={'input_tokens': 36, 'output_tokens': 23, 'total_tokens': 59})

In [19]:
from typing import TypedDict, List
from typing_extensions import Annotated

class AssessmentOutput(TypedDict):
    """Assessment output containing individual KPIs, recommended packages, and a summary report."""
    stress_management: Annotated[float, "Numerical score for Stress Management KPI"]
    therapy_engagement: Annotated[float, "Numerical score for Therapy Engagement KPI"]
    motivation: Annotated[float, "Numerical score for Motivation KPI"]
    restless_night_score: Annotated[float, "Numerical score for Restless Night Score KPI"]
    anxiety_level: Annotated[float, "Numerical score for Anxiety Level KPI"]
    burnout_level: Annotated[float, "Numerical score for Burnout Level KPI"]
    balanced_weight: Annotated[float, "Numerical score for Balanced Weight KPI"]
    package_focus: Annotated[str, "Category for package recommendation: 'Focus', 'Nutrition', or 'Yoga/Physical'"]
    report: Annotated[str, "Text-based summary report of the assessment results and reasoning behind recommendations"]


In [16]:
from langchain.schema import SystemMessage, HumanMessage
from langchain.prompts import PromptTemplate

In [15]:
template = """
You are an AI wellness assessment system. Based on the user's responses, predict the following KPIs:
- stress_management
- motivation
- restless_night_score
- anxiety_level
- burnout_level
- balanced_weight
- package_focus (Focus, Nutrition, or Fitness)
- report (a summary of the assessment results and recommendations)

### Assessment Questions:

1. How often do you feel stressed by daily tasks?
   Options: Never, Sometimes, Often, Always
   Answer: {ans1}

2. How good are you at handling stress?
   Options: Excellent, Good, Fair, Poor
   Answer: {ans2}

3. How confident are you in the ways you deal with stress?
   Options: Confident, Somewhat Confident, Not at all
   Answer: {ans3}

4. How motivated are you to work on your wellness goals daily?
   Options: Very motivated, Moderately motivated, Slightly motivated, Not at all
   Answer: {ans4}

5. How much energy do you have when you start your day to work on your goals?
   Options: Full energy, Some energy, Very little energy, None
   Answer: {ans5}

6. How often do you have trouble falling asleep?
   Options: Never, Sometimes, Often, Always
   Answer: {ans6}

7. How would you rate the quality of your sleep overall?
   Options: Excellent, Good, Fair, Poor
   Answer: {ans7}

8. How often do you wake up during the night?
   Options: Never, Sometimes, Often, Always
   Answer: {ans8}

9. Do you feel refreshed when you wake up in the morning?
   Options: Always, Most of the time, Sometimes, Never
   Answer: {ans9}

10. How often do you feel physical signs of anxiety during your daily activities?
    Options: Never, Sometimes, Often, Always
    Answer: {ans10}

11. How easy can you calm down when you're feeling anxious?
    Options: Very easy, Neutral, Somewhat difficult, Very difficult
    Answer: {ans11}

12. How often do you feel nervous for no apparent reason?
    Options: Never, Sometimes, Often, Always
    Answer: {ans12}

13. How often do you feel emotionally drained from work or your daily responsibilities?
    Options: Never, Sometimes, Often, Always
    Answer: {ans13}

14. How would you rate your current level of burnout?
    Options: Not at all, Mild, Moderate, Severe
    Answer: {ans14}

15. How often do you feel uninterested in your daily tasks?
    Options: Never, Sometimes, Often, Always
    Answer: {ans15}

16. Do you feel like your workload is too much to handle?
    Options: Never, Sometimes, Often, Always
    Answer: {ans16}

17. Which wellness area do you think needs the most improvement?
    Options: Focus (mental clarity & concentration), Nutrition (diet and weight management), Fitness (physical activity and flexibility)
    Answer: {ans17}

18. How happy are you with your current eating habits?
    Options: Satisfied, Neutral, Unsatisfied, Very unsatisfied
    Answer: {ans18}

19. How often do you do physical activities like yoga, stretching, or exercise?
    Options: Daily, Several times a week, Once a week, Never
    Answer: {ans19}

20. How would you rate your ability to stay focused and concentrate on daily tasks?
    Options: Excellent, Good, Fair, Poor
    Answer: {ans20}

Use this information to generate KPI predictions and a summary report.
"""

# Define the LangChain prompt template
prompt_template = PromptTemplate(
    input_variables=[
        "ans1", "ans2", "ans3", "ans4", "ans5", "ans6", "ans7", "ans8", "ans9", "ans10",
        "ans11", "ans12", "ans13", "ans14", "ans15", "ans16", "ans17", "ans18", "ans19", "ans20"
    ],
    template=template
)

# Sample responses
sample_inputs = {
    "ans1": "Often",
    "ans2": "Good",
    "ans3": "Confident",
    "ans4": "Very motivated",
    "ans5": "Full energy",
    "ans6": "Sometimes",
    "ans7": "Good",
    "ans8": "Sometimes",
    "ans9": "Most of the time",
    "ans10": "Sometimes",
    "ans11": "Neutral",
    "ans12": "Never",
    "ans13": "Sometimes",
    "ans14": "Moderate",
    "ans15": "Sometimes",
    "ans16": "Sometimes",
    "ans17": "Nutrition",
    "ans18": "Satisfied",
    "ans19": "Several times a week",
    "ans20": "Good"
}

# Generate formatted prompt
formatted_prompt = prompt_template.format(**sample_inputs)

In [20]:
structured_llm = chat.with_structured_output(AssessmentOutput)

In [22]:
response = structured_llm.invoke(formatted_prompt)

In [23]:
response

{'anxiety_level': 4,
 'balanced_weight': 6,
 'burnout_level': 5,
 'motivation': 8,
 'package_focus': 'Nutrition',
 'report': 'Based on your assessment, you are experiencing moderate levels of anxiety and burnout. However, you are highly motivated to work on your wellness goals and have good energy levels. Your sleep quality is good, but you sometimes have trouble falling asleep or wake up during the night. You are satisfied with your current eating habits, but feel that nutrition is the area that needs the most improvement. It is recommended that you focus on nutrition and consider seeking professional help to manage stress and anxiety. Engaging in physical activities like yoga or exercise several times a week can also help improve your overall well-being.',
 'restless_night_score': 3,
 'stress_management': 7,
 'therapy_engagement': 6}

In [24]:
type(response['motivation'])

int